[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/acb-code/nautilus/blob/main/notebooks/colab_cartpole.ipynb)

# Nautilus PPO on CartPole (Google Colab)

This notebook shows a minimal end-to-end run of Nautilus PPO on CartPole using a Colab GPU.
It installs dependencies, clones the repo, and runs the PPO runner with TensorBoard logging.

## 1. Runtime setup
- In Colab, set runtime to GPU (Runtime → Change runtime type → GPU).
- Restart runtime after installing if prompted.

In [ ]:
# Check GPU availability
!nvidia-smi

## 2. Install Nautilus
We install directly from the current repo. If you forked it, replace the URL.
This will also pull PyTorch CPU/CUDA wheels; Colab's default image already has CUDA drivers.

In [ ]:
%%bash
set -euo pipefail
# Install Nautilus from GitHub with dev extras
pip install --upgrade pip
pip install "nautilus-rl[dev] @ git+https://github.com/acb-code/nautilus.git"

# Optional: install wandb for tracking
# pip install wandb


## 3. Quick sanity import
Verify the package imports correctly.

In [ ]:
import torch

print("PyTorch:", torch.__version__, "CUDA available:", torch.cuda.is_available())

## 4. Train PPO on CartPole
This runs the PPO runner for a small number of steps to keep Colab fast.
Adjust `--total-steps` / `--num-envs` to experiment.

In [ ]:
%%bash
python -m nautilus.runners.ppo_runner \
  --env-id CartPole-v1 \
  --total-steps 50000 \
  --num-envs 4 \
  --lr 3e-4 \
  --seed 1
# To enable Weights & Biases tracking, add: --track --wandb-project-name nautilus-colab-demo


## 5. Tracking (optional)

If you want to track runs with Weights & Biases, first run `import wandb; wandb.login()` in a new cell and paste your API key. Then rerun the training cell with the `--track --wandb-project-name <project>` flags appended.


## 6. View logs
Launch TensorBoard in Colab. Click the generated link to open the UI.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

## 7. Plot episodic return
Use matplotlib to read the latest TensorBoard run and plot `charts/episodic_return`.


In [ ]:
%pip install -q matplotlib tensorboard

from pathlib import Path

import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator

run_dirs = sorted(Path("runs").glob("*"), key=lambda p: p.stat().st_mtime, reverse=True)
if not run_dirs:
    raise SystemExit("No runs found. Run the training cell first.")
latest = run_dirs[0]
ea = event_accumulator.EventAccumulator(str(latest))
ea.Reload()
scalars = ea.Scalars("charts/episodic_return")
steps = [s.step for s in scalars]
vals = [s.value for s in scalars]
plt.plot(steps, vals)
plt.xlabel("Steps")
plt.ylabel("Episodic Return")
plt.title(f"Returns for {latest.name}")
plt.grid(True)
plt.show()

## 8. Notes
- If WandB is enabled, run `wandb.login()` in a cell and paste your API key when prompted.
- Increase `--total-steps` for longer training or switch `--env-id` to try other Gymnasium envs.
- The runner writes checkpoints to `checkpoints/` if `save_interval` is reached.
